# PRÁCTICA 2A: Fine-tuning Eficiente con LoRA

**Objetivo**: Aprender técnicas de Parameter-Efficient Fine-Tuning (PEFT) con LoRA.

En este notebook exploraremos:
- **LoRA (Low-Rank Adaptation)**: Técnica de fine-tuning eficiente
- **PEFT**: Parameter-Efficient Fine-Tuning concepts
- **Comparación**: Entrenar solo ~1% de parámetros vs entrenar todo

Métricas a observar:
- Número de parámetros entrenables con LoRA
- Tiempo de entrenamiento
- Uso de memoria
- Accuracy final

## 1. Setup y Preparación del Dataset (5 min)

**Requisitos previos:**
- Python 3.8+
- Paquetes: `transformers`, `datasets`, `peft`, `evaluate`, `torch`
- Opcional: GPU con CUDA para acelerar (funciona también en CPU)

**Nota**: Este notebook usa un subset pequeño del dataset (1000 ejemplos) para demostración rápida.
Para resultados de producción, usa el dataset completo.

### Nota Importante sobre GPUs Múltiples

Si tienes múltiples GPUs y experimentas errores NCCL, ejecuta esto **ANTES** de abrir el notebook:
```bash
export CUDA_VISIBLE_DEVICES=0
```

O reinicia el kernel después de ejecutar la primera celda.


In [ ]:
!pip install evaluate

In [ ]:
# Imports
import torch
import numpy as np
import time
import os

# IMPORTANTE: Configurar para usar solo 1 GPU ANTES de importar transformers
# Esto previene errores NCCL en sistemas con múltiples GPUs
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Para debugging si es necesario

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model, TaskType
import evaluate
import warnings
warnings.filterwarnings('ignore')

# Verificar GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria disponible: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"Número de GPUs visibles: {torch.cuda.device_count()}")

### 1.1 Cargar y Preparar el Dataset

Usaremos el dataset **IMDB** para clasificación de sentimiento (positivo/negativo).

https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/data

In [2]:
!pip install datasets

  Using cached pyarrow-22.0.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.6.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.18-py312-none-any.whl.metadata (7.5 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-win_amd64.whl.metadata (2.4 kB)
  Using cached aiohttp-3.13.2-cp312-cp312-win_amd64.whl.metadata (8.4 kB)
  Using cached anyio-4.11.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached certifi-2025.10.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Cargar dataset IMDB
from datasets import load_dataset

print("Cargando dataset IMDB...")
dataset = load_dataset("imdb")

print(f"\nEstructura del dataset:")
print(dataset)

print(f"\nEjemplo de muestra:")
print(f"Texto: {dataset['train'][0]['text'][:200]}...")
print(f"Label: {dataset['train'][0]['label']} (0=negativo, 1=positivo)")

# Estadísticas
print(f"\nEstadísticas:")
print(f"Train: {len(dataset['train'])} ejemplos")
print(f"Test: {len(dataset['test'])} ejemplos")

c:\Universidad\1\Universidad\TFG\TFG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cargando dataset IMDB...


c:\Universidad\1\Universidad\TFG\TFG\.venv\Lib\site-packages\huggingface_hub\file_download.py:121: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pablo\.cache\huggingface\hub\datasets--imdb. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 273662.81 examples/s]



Estructura del dataset:
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Ejemplo de muestra:
Texto: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ev...
Label: 0 (0=negativo, 1=positivo)

Estadísticas:
Train: 25000 ejemplos
Test: 25000 ejemplos


In [ ]:
# Para acelerar el entrenamiento, usaremos un subset pequeño
# Suficiente para demostrar el concepto y completar en ~5-10 minutos

TRAIN_SIZE = 1000  # Número de ejemplos para entrenamiento (reducido para rapidez)
TEST_SIZE = 300    # Número de ejemplos para evaluación

# Crear subsets con shuffle para diversidad
train_dataset = dataset['train'].shuffle(seed=42).select(range(TRAIN_SIZE))
test_dataset = dataset['test'].shuffle(seed=42).select(range(TEST_SIZE))

print(f"Usando subset reducido:")
print(f"Train: {len(train_dataset)} ejemplos")
print(f"Test: {len(test_dataset)} ejemplos")
print(f"\nNota: Para producción usarías el dataset completo (25K ejemplos)")
print(f"      Aquí priorizamos rapidez de demostración")

### 1.2 Tokenización

In [ ]:
# Cargar tokenizer
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Función de tokenización
def tokenize_function(examples):
    """
    Tokeniza los textos del dataset.
    
    Parámetros importantes:
    - padding="max_length": Rellena todas las secuencias a la misma longitud (max_length)
                            Alternativas: "longest" (rellena al más largo del batch)
    - truncation=True: Corta secuencias más largas que max_length
    - max_length=512: Longitud máxima de la secuencia en tokens
                      DistilBERT admite hasta 512 tokens
    """
    return tokenizer(
        examples["text"],           # Textos a tokenizar
        padding="max_length",       # Rellenar todas las secuencias a max_length
        truncation=True,            # Cortar si excede max_length
        max_length=512              # Longitud máxima (límite del modelo)
    )

# Tokenizar datasets
print("Tokenizando datasets...")
tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_test = test_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

print("\nDatasets tokenizados:")
print(f"Train: {tokenized_train}")
print(f"Test: {tokenized_test}")

# DataCollatorWithPadding

DataCollatorWithPadding es un componente de Hugging Face Transformers que prepara los batches de datos durante el entrenamiento. Veamos qué hace y sus alternativas:

¿Qué hace DataCollatorWithPadding?
Función principal: Aplica padding dinámico a las secuencias dentro de cada batch para que todas tengan la misma longitud.

Cómo funciona:

Recibe un batch de ejemplos tokenizados
Encuentra la secuencia más larga del batch
Rellena (pad) todas las demás secuencias a esa longitud
Crea las máscaras de atención automáticamente

In [ ]:
# Data collator para padding dinámico
# Alternativa a padding="max_length": permite diferentes longitudes por batch
# Más eficiente en memoria cuando las secuencias tienen longitudes variadas
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,     # Tokenizer para aplicar padding
    padding=True,            # Aplicar padding dinámico
    max_length=None,         # Sin límite adicional (usa el del tokenizer)
    pad_to_multiple_of=None  # Sin redondeo de longitud
)

# Métrica de evaluación
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """
    Calcula métricas de evaluación durante el entrenamiento.
    
    Args:
        eval_pred: Tuple con (predictions, labels)
                   predictions: logits del modelo (shape: [batch_size, num_labels])
                   labels: etiquetas verdaderas (shape: [batch_size])
    
    Returns:
        Dict con métricas calculadas
    """
    predictions, labels = eval_pred
    # Convertir logits a clases predichas (argmax sobre dimensión de clases)
    predictions = np.argmax(predictions, axis=1)
    # Calcular accuracy comparando predicciones con labels verdaderos
    return accuracy_metric.compute(predictions=predictions, references=labels)

## 2. Full Fine-tuning: Entrenar Todos los Parámetros

**Primero entrenaremos el modelo completo** para establecer una línea base de comparación.

En full fine-tuning:
- Entrenamos **todos** los parámetros del modelo (~67M para DistilBERT)
- Máxima capacidad de adaptación
- Alto uso de memoria
- Más lento
- Requiere más datos para evitar overfitting

**Métricas a observar:**
- Memoria GPU usada durante el entrenamiento
- Tiempo de entrenamiento
- Accuracy final


In [ ]:
print("=" * 70)
print("FULL FINE-TUNING (Entrenar todos los parámetros)")
print("=" * 70)

# Cargar modelo base (sin LoRA)
model_full = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
).to(device)

# Contar parámetros
total_params_full = sum(p.numel() for p in model_full.parameters())
trainable_params_full = sum(p.numel() for p in model_full.parameters() if p.requires_grad)

print(f"\nInformación del modelo:")
print(f"Parámetros totales: {total_params_full:,}")
print(f"Parámetros entrenables: {trainable_params_full:,}")
print(f"Porcentaje entrenable: {100 * trainable_params_full / total_params_full:.2f}%")
print(f"\n¡Entrenamos el 100% de los parámetros!")

In [ ]:
# Configuración de entrenamiento (misma que usaremos con LoRA para comparar)
training_args_full = TrainingArguments(
    output_dir="./results_full",
    num_train_epochs=3,              # Número de épocas
    per_device_train_batch_size=8,   # Batch size (igual que LoRA)
    per_device_eval_batch_size=16,   # Batch size para evaluación
    learning_rate=2e-5,              # Learning rate (más bajo que LoRA, típico para full FT)
    weight_decay=0.01,               # Regularización L2
    eval_strategy="epoch",           # Evaluar al final de cada época
    save_strategy="no",              # No guardar checkpoints
    logging_steps=50,                # Log cada 50 steps
    report_to="none",                # No reportar a wandb/tensorboard
    fp16=torch.cuda.is_available(),  # Mixed precision para ahorrar memoria
)

# Crear Trainer
trainer_full = Trainer(
    model=model_full,
    args=training_args_full,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("\nTrainer configurado. Listo para entrenar.")

In [ ]:
# Entrenar y medir recursos
torch.cuda.empty_cache() if torch.cuda.is_available() else None

# Métricas de memoria
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()
    start_memory = torch.cuda.memory_allocated() / 1e9  # GB
    print(f"Memoria GPU inicial: {start_memory:.2f} GB")

# Medir tiempo
start_time = time.time()

print("\nIniciando entrenamiento FULL FINE-TUNING...")
print("   (Esto puede tardar ~5-10 minutos)\n")

trainer_full.train()

end_time = time.time()
training_time_full = end_time - start_time

# Reportar uso de memoria
if torch.cuda.is_available():
    peak_memory_full = torch.cuda.max_memory_allocated() / 1e9  # GB
    current_memory = torch.cuda.memory_allocated() / 1e9  # GB
    print(f"\nREPORTE DE MEMORIA:")
    print(f"   Memoria inicial: {start_memory:.2f} GB")
    print(f"   Memoria actual: {current_memory:.2f} GB")
    print(f"   Memoria pico durante entrenamiento: {peak_memory_full:.2f} GB")
    print(f"   Memoria adicional usada: {peak_memory_full - start_memory:.2f} GB")
else:
    peak_memory_full = 0
    print(f"\nEjecutando en CPU (no hay métricas de memoria GPU)")

print(f"\nTIEMPO DE ENTRENAMIENTO:")
print(f"   {training_time_full:.2f} segundos ({training_time_full/60:.2f} minutos)")


In [ ]:
# Evaluar el modelo
print("\nEvaluando modelo con Full Fine-tuning...")
eval_results_full = trainer_full.evaluate()

print("\n" + "=" * 70)
print("RESULTADOS - FULL FINE-TUNING")
print("=" * 70)
print(f"Accuracy: {eval_results_full['eval_accuracy']:.4f}")
print(f"Loss: {eval_results_full['eval_loss']:.4f}")
print(f"Parámetros entrenables: {trainable_params_full:,} (100%)")
print(f"Tiempo: {training_time_full:.2f}s ({training_time_full/60:.2f} min)")
if torch.cuda.is_available():
    print(f"Memoria pico: {peak_memory_full:.2f} GB")
print("=" * 70)

# Limpiar memoria para LoRA
del model_full, trainer_full
torch.cuda.empty_cache() if torch.cuda.is_available() else None
print("\nModelo full fine-tuning completado. Memoria liberada para LoRA.")


## 3. LoRA: Parameter-Efficient Fine-Tuning

**Ahora veremos LoRA** - una técnica que permite entrenar modelos grandes con pocos recursos.
En lugar de entrenar todos los parámetros, añadimos matrices pequeñas de bajo rango.

### ¿Cómo funciona LoRA?

En lugar de actualizar directamente las matrices de peso $W$, LoRA añade dos matrices pequeñas $A$ y $B$:

$$W' = W + \Delta W = W + BA$$

Donde:
- $W \in \mathbb{R}^{d \times k}$ (matriz original, **congelada**)
- $B \in \mathbb{R}^{d \times r}$ (matriz entrenable)
- $A \in \mathbb{R}^{r \times k}$ (matriz entrenable)
- $r \ll \min(d, k)$ (rank pequeño, típicamente 4-16)

### Parámetros Clave de LoRA:

1. **`r` (rank)**: Dimensión del espacio de bajo rango
   - Valores típicos: 4, 8, 16, 32
   - Mayor rank = más parámetros entrenables = mejor capacidad pero más costoso
   - Recomendado: empezar con 8

2. **`lora_alpha`**: Factor de escalado
   - Típicamente `2*r` o `4*r`
   - Controla la magnitud de las actualizaciones de LoRA
   - No afecta el número de parámetros

3. **`target_modules`**: Qué capas modificar con LoRA
   - `["q_lin", "v_lin"]`: Solo query y value (común en transformers)
   - `["q_lin", "v_lin", "k_lin", "out_lin"]`: Todas las proyecciones de atención
   - Más módulos = más parámetros entrenables

4. **`lora_dropout`**: Dropout aplicado a las capas LoRA
   - Típicamente 0.05-0.1
   - Ayuda a prevenir overfitting

**Ventajas**:
- Solo entrenamos ~0.1-1% de parámetros
- Mucho más rápido y eficiente en memoria
- Ideal para múltiples tareas (puedes tener diferentes adaptadores LoRA)
- Fácil de compartir y combinar adaptadores


In [ ]:
print("=" * 70)
print("ENTRENAMIENTO CON LORA (PEFT)")
print("=" * 70)

# Cargar modelo base
model_lora = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
).to(device)

# Configurar LoRA
lora_config = LoraConfig(
    r=8,                                    # Rank: dimensión del espacio de bajo rango (4-32)
    lora_alpha=32,                          # Scaling factor: típicamente 2*r o 4*r
    target_modules=["q_lin", "v_lin"],      # Módulos a adaptar: query y value attention
    lora_dropout=0.1,                       # Dropout para regularización
    bias="none",                            # No entrenar bias adicionales
    task_type=TaskType.SEQ_CLS              # Tipo de tarea: Sequence Classification
)

# Aplicar LoRA al modelo
model_lora = get_peft_model(model_lora, lora_config)

# Imprimir información del modelo
print("\nInformación del modelo con LoRA:")
model_lora.print_trainable_parameters()

# Contar parámetros manualmente para comparación
total_params_lora = sum(p.numel() for p in model_lora.parameters())
trainable_params_lora = sum(p.numel() for p in model_lora.parameters() if p.requires_grad)

print(f"\nRESUMEN:")
print(f"Parámetros totales: {total_params_lora:,}")
print(f"Parámetros entrenables: {trainable_params_lora:,}")
print(f"Porcentaje entrenable: {100 * trainable_params_lora / total_params_lora:.2f}%")
print(f"\n¡Solo entrenamos ~1% de los parámetros del modelo!")
print(f"\nCon rank r={lora_config.r}:")
print(f"  - Cada matriz LoRA añade: d×r + r×k parámetros")
print(f"  - Para attention de dim 768: ~{(768*8 + 8*768)*2:,} params por capa")

In [ ]:
# Configuración de entrenamiento
training_args_lora = TrainingArguments(
    output_dir="./results_lora",
    num_train_epochs=3,              # Número de épocas completas de entrenamiento
    per_device_train_batch_size=8,   # Batch size para entrenamiento
    per_device_eval_batch_size=16,   # Batch size para evaluación (puede ser mayor)
    learning_rate=2e-4,              # Learning rate (típicamente más alto con LoRA: 1e-4 a 3e-4)
    weight_decay=0.01,               # Regularización L2
    eval_strategy="epoch",           # Evaluar al final de cada época
    save_strategy="no",              # No guardar checkpoints (para rapidez)
    logging_steps=50,                # Log cada 50 steps
    report_to="none",                # No reportar a wandb/tensorboard
    fp16=torch.cuda.is_available(),  # Mixed precision training si hay GPU
)

# Crear Trainer
trainer_lora = Trainer(
    model=model_lora,
    args=training_args_lora,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
# Entrenar y medir tiempo/memoria
torch.cuda.empty_cache() if torch.cuda.is_available() else None

# Métricas de memoria
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()
    start_memory_lora = torch.cuda.memory_allocated() / 1e9  # GB
    print(f"Memoria GPU inicial: {start_memory_lora:.2f} GB")

start_time = time.time()

print("\nIniciando entrenamiento con LoRA...")
print("   (Esto debería ser más rápido que full fine-tuning)\n")

trainer_lora.train()

end_time = time.time()
training_time_lora = end_time - start_time

# Reportar uso de memoria
if torch.cuda.is_available():
    peak_memory_lora = torch.cuda.max_memory_allocated() / 1e9  # GB
    current_memory_lora = torch.cuda.memory_allocated() / 1e9  # GB
    print(f"\nREPORTE DE MEMORIA:")
    print(f"   Memoria inicial: {start_memory_lora:.2f} GB")
    print(f"   Memoria actual: {current_memory_lora:.2f} GB")
    print(f"   Memoria pico durante entrenamiento: {peak_memory_lora:.2f} GB")
    print(f"   Memoria adicional usada: {peak_memory_lora - start_memory_lora:.2f} GB")
else:
    peak_memory_lora = 0
    print(f"\nEjecutando en CPU (no hay métricas de memoria GPU)")

print(f"\nTIEMPO DE ENTRENAMIENTO:")
print(f"   {training_time_lora:.2f} segundos ({training_time_lora/60:.2f} minutos)")


In [ ]:
# Evaluar
print("\nEvaluando modelo con LoRA...")
eval_results_lora = trainer_lora.evaluate()

print("\n" + "=" * 70)
print("RESULTADOS - LORA (PEFT)")
print("=" * 70)
print(f"Accuracy: {eval_results_lora['eval_accuracy']:.4f}")
print(f"Loss: {eval_results_lora['eval_loss']:.4f}")
print(f"Parámetros entrenables: {trainable_params_lora:,} ({100 * trainable_params_lora / total_params_lora:.2f}%)")
print(f"Tiempo: {training_time_lora:.2f}s ({training_time_lora/60:.2f} min)")
if torch.cuda.is_available():
    print(f"Memoria pico: {peak_memory_lora:.2f} GB")
print("=" * 70)

print("\nLoRA permite hacer fine-tuning de modelos grandes eficientemente")
print("   Ideal para: recursos limitados, múltiples tareas, experimentación rápida")


### 3.1 Comparación Directa: Full Fine-tuning vs LoRA

Comparemos los resultados lado a lado:

In [ ]:
import pandas as pd

# Crear tabla comparativa
comparison_data = {
    'Métrica': [
        'Accuracy',
        'Loss',
        'Parámetros Entrenables',
        '% Parámetros',
        'Tiempo (segundos)',
        'Tiempo (minutos)',
        'Memoria Pico (GB)',
        'Speedup vs Full FT',
        'Ahorro Memoria vs Full FT'
    ],
    'Full Fine-tuning': [
        f"{eval_results_full['eval_accuracy']:.4f}",
        f"{eval_results_full['eval_loss']:.4f}",
        f"{trainable_params_full:,}",
        "100.00%",
        f"{training_time_full:.2f}",
        f"{training_time_full/60:.2f}",
        f"{peak_memory_full:.2f}" if torch.cuda.is_available() else "N/A (CPU)",
        "1.00x (baseline)",
        "0% (baseline)"
    ],
    'LoRA': [
        f"{eval_results_lora['eval_accuracy']:.4f}",
        f"{eval_results_lora['eval_loss']:.4f}",
        f"{trainable_params_lora:,}",
        f"{100 * trainable_params_lora / total_params_lora:.2f}%",
        f"{training_time_lora:.2f}",
        f"{training_time_lora/60:.2f}",
        f"{peak_memory_lora:.2f}" if torch.cuda.is_available() else "N/A (CPU)",
        f"{training_time_full/training_time_lora:.2f}x" if training_time_lora > 0 else "N/A",
        f"{100*(peak_memory_full - peak_memory_lora)/peak_memory_full:.1f}%" if torch.cuda.is_available() and peak_memory_full > 0 else "N/A"
    ]
}

df_comparison = pd.DataFrame(comparison_data)

print("\n" + "=" * 80)
print("COMPARACIÓN: FULL FINE-TUNING vs LoRA")
print("=" * 80)
print(df_comparison.to_string(index=False))
print("=" * 80)

# Análisis de resultados
print("\nANÁLISIS:")
print(f"\n1. PRECISIÓN:")
acc_diff = eval_results_lora['eval_accuracy'] - eval_results_full['eval_accuracy']
if abs(acc_diff) < 0.01:
    print(f"   Accuracy similar entre ambos métodos (diferencia: {acc_diff:+.4f})")
else:
    print(f"   Accuracy diferente: {acc_diff:+.4f}")

print(f"\n2. EFICIENCIA DE PARÁMETROS:")
param_reduction = 100 * (1 - trainable_params_lora / trainable_params_full)
print(f"   LoRA entrena {param_reduction:.1f}% menos parámetros")
print(f"   Solo {trainable_params_lora:,} parámetros vs {trainable_params_full:,}")

print(f"\n3. VELOCIDAD:")
if training_time_lora > 0:
    speedup = training_time_full / training_time_lora
    time_saved = training_time_full - training_time_lora
    print(f"   LoRA es {speedup:.2f}x más rápido")
    print(f"   Ahorro de tiempo: {time_saved:.1f} segundos ({time_saved/60:.1f} minutos)")

if torch.cuda.is_available() and peak_memory_full > 0:
    print(f"\n4. MEMORIA:")
    memory_saved = peak_memory_full - peak_memory_lora
    memory_reduction = 100 * memory_saved / peak_memory_full
    print(f"   LoRA usa {memory_reduction:.1f}% menos memoria")
    print(f"   Ahorro: {memory_saved:.2f} GB")
    print(f"   Esto permite entrenar modelos más grandes en la misma GPU")

print(f"\nCONCLUSIÓN:")
print(f"   LoRA ofrece un excelente balance entre eficiencia y rendimiento.")
print(f"   Es ideal cuando tienes recursos limitados o necesitas entrenar rápido.")


## 4. Experimentando con LoRA

### 4.1 Efecto del Rank (r)

El parámetro más importante en LoRA es el **rank** ($r$). Veamos cómo afecta:

In [ ]:
# Comparar diferentes ranks
ranks = [4, 8, 16]

print("=" * 70)
print("COMPARACIÓN DE RANKS EN LORA")
print("=" * 70)

for r in ranks:
    # Crear modelo con LoRA
    model_temp = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=2
    )
    
    lora_config_temp = LoraConfig(
        r=r,
        lora_alpha=32,
        target_modules=["q_lin", "v_lin"],
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.SEQ_CLS
    )
    
    model_temp = get_peft_model(model_temp, lora_config_temp)
    
    total_params = sum(p.numel() for p in model_temp.parameters())
    trainable_params = sum(p.numel() for p in model_temp.parameters() if p.requires_grad)
    
    print(f"\nRank r={r}:")
    print(f"  Parámetros entrenables: {trainable_params:,}")
    print(f"  Porcentaje: {100 * trainable_params / total_params:.2f}%")
    
    del model_temp

print("\n" + "=" * 70)
print("\nObservaciones:")
print("  - Ranks más altos = más parámetros entrenables")
print("  - Pero incluso con r=16, seguimos en ~2% de parámetros")
print("  - En la práctica: r=4-8 suele ser suficiente")
print("  - r muy alto puede causar overfitting en datasets pequeños")

## 5. Ejercicios Propuestos

### Ejercicio 1: Experimentar con Diferentes Ranks
Entrena modelos con r=4, r=8 y r=16. Compara:
- Accuracy final
- Tiempo de entrenamiento
- Memoria usada

### Ejercicio 2: Más Módulos Target
Prueba aplicar LoRA a más capas:
```python
target_modules=["q_lin", "v_lin", "k_lin", "out_lin"]
```
¿Cómo afecta al rendimiento?

### Ejercicio 3: Entrenar con Dataset Completo
Entrena con los 25K ejemplos completos del dataset IMDB:
- ¿Mejora el accuracy?
- ¿Cuánto tiempo toma?
- ¿Sigue siendo eficiente LoRA?

### Ejercicio 4: Otros Modelos
Prueba LoRA con otros modelos:
- `bert-base-uncased`
- `roberta-base`
- Modelos más grandes si tienes GPU

### Ejercicio 5: Datasets Diferentes
Aplica LoRA a otros tasks:
- `sst2` (sentimiento)
- `mrpc` (paráfrasis)
- `cola` (aceptabilidad gramatical)

## 6. Resumen

### ¿Qué aprendimos?

En este notebook comparamos **Full Fine-tuning** vs **LoRA (Low-Rank Adaptation)**:

**Full Fine-tuning**:
- Máxima capacidad de adaptación
- Alto uso de memoria (varios GB)
- Más lento
- Requiere más recursos

**LoRA (Low-Rank Adaptation)**:
- Entrena solo ~1% de parámetros
- **Eficiencia**: Entrenar solo ~1% de los parámetros del modelo
- **Velocidad**: 3-5x más rápido que full fine-tuning
- **Memoria**: Usa ~50-70% menos memoria
- **Modularidad**: Múltiples adaptadores para diferentes tareas

### Parámetros Críticos para Recordar:

| Parámetro | Valores Típicos | Efecto |
|-----------|----------------|--------|
| **r** (rank) | 4, 8, 16 | Mayor = más capacidad, más parámetros |
| **lora_alpha** | 16, 32, 64 | Típicamente 2*r o 4*r |
| **target_modules** | ["q_lin", "v_lin"] | Qué capas adaptar |
| **learning_rate** | 1e-4 a 3e-4 | Mayor que full fine-tuning |

### Cuándo usar LoRA:

**SÍ usar LoRA cuando:**
- Recursos limitados (GPU pequeña o CPU)
- Necesitas entrenar rápidamente
- Múltiples tareas/dominios con el mismo modelo base
- Experimentación rápida con diferentes configuraciones

**NO usar LoRA cuando:**
- Tienes recursos ilimitados y necesitas máximo rendimiento
- La tarea es muy diferente del pretraining
- Dataset extremadamente grande y específico

### Próximos Pasos:

1. **Experimenta** con diferentes ranks (r=4, 8, 16, 32)
2. **Prueba** más target_modules
3. **Compara** con full fine-tuning en tu tarea específica
4. **Explora** QLoRA para modelos aún más grandes


## 7. Recursos Adicionales

### Papers Importantes:
- **LoRA**: [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685)
- **QLoRA**: [QLoRA: Efficient Finetuning of Quantized LLMs](https://arxiv.org/abs/2305.14314)
- **Prefix Tuning**: [Prefix-Tuning: Optimizing Continuous Prompts](https://arxiv.org/abs/2101.00190)

### Librerías:
- **Hugging Face PEFT**: https://github.com/huggingface/peft
- **Documentación PEFT**: https://huggingface.co/docs/peft

### Investiga otras técnicas PEFT:
- **Adapter Layers**: Añadir capas pequeñas entre layers
- **Prompt Tuning**: Solo entrenar los embeddings del prompt
- **QLoRA**: LoRA + 4-bit quantization

Creado por Jorge Dueñas Lerín
jorge.duenas.lerin@upm.es